## Create NetCDF from .csv file

Experiement to use metsim (python pacage) to convert HADS gage time series data into NetCDF format

In [ ]:
#!conda install -c conda-forge cartopy --y

In [ ]:
#!conda install -c conda-forge geoviews --y

In [ ]:
#!conda install -c conda-forge metsim --y

begin with some standard imports.

In [ ]:
#!conda update matplotlib --y

In [ ]:
%pylab inline
import cartopy
import geoviews as gv
import geopandas as gpd
import holoviews as hv
import pandas as pd
import xarray as xr
from metsim import MetSim

pylab.rcParams['figure.figsize'] = (10,6)
plt.rcParas['figure.dpi'] = 96
hv.notebook_extension('bokeh')

## Put together the required meteorological data
We've gathered some data form the HADS site that we will use as input. There are different sets of CSVs with precipitation, with data for 2009 and 2010, and will be generating MetSim input for 2010. To do this we must first convert it into an xarray dataset.

HADS example files

In [21]:
import pandas as pd
import xarray as xr
import numpy as np

In [22]:
datameta = pd.read_csv("../example_data/HADS/DCPbySTATE_HI_station_info.csv")
data = pd.read_csv("../example_data/HADS/HADS_20190323.txt", sep="|",header=None)

In [23]:
#datameta

In [24]:
data.columns = ["nesdis_id", "nwsli", "type","datetime", "cumP", "5","6"]
#data

In [25]:
data.nesdis_id.unique()[0:2]

array(['1560029C', '156011EA'], dtype=object)

In [26]:
latlongs=pd.DataFrame()
for i in data.nesdis_id.unique()[0:2]:
    temp=datameta.loc[datameta['nesdis_id'] == i, 'latitude_d':'longitude_d']
    latlongs=latlongs.append(temp)
    
print(latlongs)


    latitude_d  longitude_d
6      22.1375     -159.312
25     21.9364     -159.587


not quite working yet
error message:
1560029C
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-20-91d6b020cfe3> in <module>()
     57                 threehrP=sumeveryxrows(temp,3)
     58                 threehrP=np.append(threehrP,sum(hrP[-mod:]))
---> 59         threehrP=np.append(sum(temp1),threehrP)
     60     #print(threehrP)

TypeError: 'numpy.float64' object is not iterable

In [20]:
sumeveryxrows = lambda myarray, x: myarray.values.reshape((myarray.shape[0] // x, x)).sum(1)
from datetime import datetime
for i in data.nesdis_id.unique()[0:2]:
    print(i)
    #i=data.nesdis_id.unique()[0]
    sitedata = data.loc[(data.nesdis_id == i),:]
    cumP_tnp1= np.append([np.nan],sitedata.cumP)
    unitP=sitedata.cumP-cumP_tnp1[0:-1]
    sitedata = pd.concat([sitedata, unitP], axis=1)
    #sitedata.columns = ['unitP']
    sitedata.columns=['nesdis_id','nwsli','type','datetime','cumP','5','6','unitP']
    #print(sitedata)
    # Create a pandas series, using 'time' as the index
    #print(sitedata.unitP)
    #print(sitedata.datetime.to_string())
    datetime_list = [datetime.strptime(time, '%Y-%m-%d %H:%M') for time in sitedata.datetime.astype(str)]
    #print(datetime_list)
    #datetime_object = datetime.strptime((sitedata.datetime.astype(str)[0]), '%Y-%m-%d %H:%M')
    #datetime.strptime("21/11/06 16:30", "%d/%m/%y %H:%M")
    series = pd.Series(np.array(sitedata.unitP), index=datetime_list)
    hrP=series.groupby([series.index.date, series.index.hour]).sum()
    #print(hrP) 
    ## turn it into every 3 hour
    ## WRF-Hydro run by UTC0, Hawaii is (23-1,2-4,5-7,8-10,11-13,14-16,17-19,20-22)hr
    firsthr=hrP.index[0][1]
    if firsthr==23|firsthr==2|firsthr==5|firsthr==8|firsthr==11|firsthr==14|firsthr==17|firsthr==20:
        if len(hrP)%3==0:
            threehrP=sumeveryxrows(hrP,3)
        else:
            mod=len(hrP)%3
            temp=hrP[0:-mod]
            threehrP=sumeveryxrows(temp,3)
            threehrP=np.append(threehrP,sum(hrP[-mod:]))
    elif firsthr==0|firsthr==3|firsthr==4|firsthr==5|firsthr==12|firsthr==15|firsthr==18|firsthr==21: #need to revise later
        temp1=hrP[0:2]
        hrP=hrP[2:]
        firsthr=hrP.index[0][1]
        if firsthr==23|firsthr==2|firsthr==5|firsthr==8|firsthr==11|firsthr==14|firsthr==17|firsthr==20:
            if len(hrP)%3==0:
                    threehrP=sumeveryxrows(hrP,3)
            else:
                mod=len(hrP)%3
                temp=hrP[0:-mod]
                threehrP=sumeveryxrows(temp,3)
                threehrP=np.append(threehrP,sum(hrP[-mod:]))
        threehrP=np.append(sum(temp1),threehrP)
    else:
        temp1=hrP[0]
        hrP=hrP[1:]
        firsthr=hrP.index[0][1]
        if firsthr==23|firsthr==2|firsthr==5|firsthr==8|firsthr==11|firsthr==14|firsthr==17|firsthr==20:
            if len(hrP)%3==0:
                    threehrP=sumeveryxrows(hrP,3)
            else:
                mod=len(hrP)%3
                temp=hrP[0:-mod]
                threehrP=sumeveryxrows(temp,3)
                threehrP=np.append(threehrP,sum(hrP[-mod:]))
        threehrP=np.append(sum(temp1),threehrP)
    #print(threehrP)

1560029C


TypeError: 'numpy.float64' object is not iterable

## create the dataset with the relevant dimensions

In [ ]:
dates = pd.date_range('3/17/2019', '3/23/2019')
shape = (len(dates), 1, 1, )
dims = ('time', 'lat', 'lon', )

# We are running only one site, at these coordinates
lats = [19.8372, 21.4319]
lons = [-155.613, -157.837]
#elev = 1706.90 # meters
coords = {'time': dates, 'lat': lats, 'lon': lons}

# Create the initial met data input data structure
met_data = xr.Dataset(coords=coords)
met_data

## create the acutal data arrays to put data into

In [ ]:
for varname in ['prec']:
    met_data[varname] = xr.DataArray(data=np.full(shape, np.nan),
                                     coords=coords, dims=dims,
                                     name=varname)

## Read in the data and put it into the dataset

In [ ]:
for i in range(len(lats)):
    for j in range(len(lons)):
        # Add the precipitation data
        df = pd.read_csv("./reynolds_creek_data/daily/precip_2010.csv", skiprows=[0,1])
        prec_vals = df[].diff().values[1:]*25.4
        met_data['prec'].values[:, i, j] = prec_vals

# This is what we have now
met_data.to_netcdf('./input/rc_forcing.nc')
met_data

## Put together the required domain

In [ ]:
# We form the domain in a similar fashion
# First, by creating the data structure
coords = {'lat': lats, 'lon': lons}
domain = xr.Dataset(coords=coords)
domain['elev'] = xr.DataArray(data=np.full((1,1,), np.nan),
                          coords=coords,
                          dims=('lat', 'lon', ))
domain['mask'] = xr.DataArray(data=np.full((1,1,), np.nan),
                          coords=coords,
                          dims=('lat', 'lon', ))

# Add the data
domain['elev'][0, 0] = elev
domain['mask'][0, 0] = 1
domain.to_netcdf('./input/rc_domain.nc')
domain

## Put together the required state

In [ ]:
# Finally, we create the state file - the dates are 90 days prior to 
# the MetSim run dates - as usual, create an empty data structure to
# read the data into
dates = pd.date_range('3/16/2019', '3/23/2019')
shape = (len(dates), 1, 1, )
dims = ('time', 'lat', 'lon', )

coords = {'time': dates, 'lat': lats, 'lon': lons}
state = xr.Dataset(coords=coords)
for varname in ['prec', 't_min', 't_max']:
    state[varname] = xr.DataArray(data=np.full(shape, np.nan),
                               coords=coords, dims=dims,
                               name=varname)

# Do precip data
df = pd.read_csv("./reynolds_creek_data/daily/precip_2009.csv", skiprows=[0,1])
prec_vals = df['PREC.I-1 (in) '].diff().values[-90:] * 25.4
state['prec'].values[:, 0, 0] = prec_vals

# And now temp data
df = pd.read_csv("./reynolds_creek_data/daily/temp_2009.csv", skiprows=[0,1])
tmin_vals = df['TMIN.D-1 (degC) '].values[-90:]
tmax_vals = df['TMAX.D-1 (degC) '].values[-90:]
state['t_min'].values[:, 0, 0] = tmin_vals
state['t_max'].values[:, 0, 0] = tmax_vals
state.to_netcdf('./input/rc_state.nc')
state

## Registering parameters and building the driver

In [ ]:
dates = pd.date_range('1/1/2010', '12/31/2010')
params = {
    'time_step'    : "60",       
    'start'        : dates[0],
    'stop'         : dates[-1],
    'forcing'      : './input/rc_forcing.nc',     
    'domain'       : './input/rc_domain.nc',
    'state'        : './input/rc_state.nc',
    'forcing_fmt'  : 'netcdf',
    'out_dir'      : './output',
    'output_prefix': 'reynolds',
    'scheduler'    : 'threading',
    'chunks'       : 
        {'lat': 1, 'lon': 1},
    'forcing_vars' : 
        {'prec' : 'prec', 't_max': 't_max', 't_min': 't_min'},
    'state_vars'   : 
        {'prec' : 'prec', 't_max': 't_max', 't_min': 't_min'},
    'domain_vars'  : 
        {'elev': 'elev', 'lat': 'lat', 'lon': 'lon', 'mask': 'mask'}
    }               

ms = MetSim(params)
ms.run()
output = ms.open_output().load()